<h1>Table of contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#load-data" data-toc-modified-id="load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>load data</a></span></li><li><span><a href="#clean-corpus" data-toc-modified-id="clean-corpus-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>clean corpus</a></span><ul class="toc-item"><li><span><a href="#dataframe-cleaning" data-toc-modified-id="dataframe-cleaning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>dataframe cleaning</a></span></li><li><span><a href="#text-cleaning" data-toc-modified-id="text-cleaning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>text cleaning</a></span></li></ul></li><li><span><a href="#downsampling" data-toc-modified-id="downsampling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>downsampling</a></span></li><li><span><a href="#Add-length" data-toc-modified-id="Add-length-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add length</a></span></li><li><span><a href="#split-corpus" data-toc-modified-id="split-corpus-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>split corpus</a></span></li></ul></div>

**Dataset**:<br>
Amazon Reviews: https://nijianmo.github.io/amazon/index.html

In [1]:
import os
import json
import gzip
from nltk import word_tokenize
import pandas as pd
from urllib.request import urlopen

from utils import clean_text, random_downsampling

# load data

In [ ]:
""" very time consuming

data = []
with gzip.open('../corpora/Electronics_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))

df = pd.DataFrame.from_dict(data)

big_corpus = df[["overall", "reviewerName", "reviewText", 
                  "summary", "verified", "vote", "reviewTime"]]
                  
big_corpus.to_csv("../corpora/amazon_reviews_electronic.csv", index=False)
"""
print("Done loading.")

# clean corpus

In [ ]:
PATH = "../corpora/amazon_reviews_electronic.csv"
MIN_YEAR = 2018
COMBINE_SUMMARY = False

In [ ]:
%%time
corpus = pd.read_csv(PATH)

In [ ]:
corpus.head()

## dataframe cleaning

In [ ]:
%%time
corpus.vote = corpus.vote.fillna(0)
corpus = corpus.fillna("")


# rename columns

corpus = corpus.rename(columns={"overall": "rating", 
                                "reviewerName": "name",
                                "reviewText": "review",
                                "reviewTime": "date"})

# no empty ratings, reviews & dates

corpus = corpus[corpus.rating != ""]
corpus = corpus[corpus.review != ""]
corpus = corpus[corpus.date != ""]


# change date

def change_date(date):
    day_month, year = str(date).split(",")
    month, day = day_month.split(" ")
    
    if len(day) <= 1:
        day = "0" + str(day)
        
    
    return f"{day}.{month}.{str(year)[-4:]}"

# only reviews since a specific date

def remove_dates(date, min_year):
    year = int(str(date)[-4:])
    if year < min_year:
        return ""
    else:
        return date
    
corpus.date = corpus.date.apply(lambda x: remove_dates(x, MIN_YEAR))
corpus = corpus[corpus.date != ""]

corpus.date = corpus.date.apply(lambda x: change_date(x))


# combine summary and review text

if COMBINE_SUMMARY:
    corpus['review'] = corpus[['summary', 'review']].apply(lambda x: " ".join(str(y) for y in x if str(y) != 'nan'), axis = 1)
del corpus["summary"]

In [ ]:
corpus.sample(10)

## text cleaning

In [ ]:
%%time
corpus.review = corpus.review.apply(clean_text)
corpus = corpus[corpus.review != ""]

In [ ]:
corpus.head()

In [ ]:
corpus.shape

In [ ]:
corpus.rating.value_counts()

# downsampling

Reduces corpus size by random downsampling.

In [ ]:
s_corpus = random_downsampling(corpus, class_col="rating", max_value = 15000)

In [ ]:
#s_corpus.to_csv("../corpora/small_amazon_reviews_electronic.csv", index=False)

In [ ]:
s_corpus.shape

# Add length

In [4]:
def length(x):
    return len(word_tokenize(x))

corpus["length"] = corpus.review.apply(length)

In [6]:
#corpus.to_csv("../corpora/small_amazon_reviews_electronic.csv", index=False)

# split corpus

In [2]:
import pandas as pd
from utils import *

In [7]:
corpus = pd.read_csv("../corpora/small_amazon_reviews_electronic.csv")

In [8]:
split_corpus(corpus, split=0.8)

/Users/jan/Desktop/informatik_programme/wordembeddings/app/utils.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[text_col] = df.apply(lambda row: word_tokenize(row[text_col]), axis=1)
